In [ ]:
import wandb
import numpy as np
from TensorKit.Layers import *
from TensorKit.Models import *
from TensorKit.Optimizers import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets.fashion_mnist import load_data
np.random.seed(42)

convert_to_one_hot = lambda y : np.eye(y.max() + 1)[y]
normalize = lambda x : (x - x.mean(axis=1, keepdims=True))/x.std(axis=1, keepdims=True)

In [ ]:
(trainX, trainy), (testX, testy) = load_data()
trainX, valX, trainy, valy = train_test_split(trainX, trainy, test_size=0.1, shuffle=True)

trainX = normalize(trainX.reshape(trainX.shape[0], -1))
testX = normalize(testX.reshape(testX.shape[0], -1))
valX = normalize(valX.reshape(valX.shape[0], -1))

trainy = convert_to_one_hot(trainy)
testy = convert_to_one_hot(testy)
valy = convert_to_one_hot(valy)

In [ ]:
def build_and_run_model(config=None):
    run = wandb.init(config=config, magic=True)
    config = wandb.config
    wandb.run.name = (
        f"hl_{config.num_hidden_layers}"
        f"_hn_{config.hidden_layer_size}"
        f"_bs_{config.batch_size}"
        f"_act_{config.activation}"
        f"_opt_{config.optimizer}"
        f"_l2_{config.l2}"
        f"_init_{config.weight_init}"
        f"_loss_{config.loss_function}"
    )

    optimizer = {
        "sgd": SGD(lr=config.lr),
        "mgd": SGD(lr=config.lr, momentum=0.9),
        "nag": SGD(lr=config.lr, momentum=0.9, nesterov=True),
        "adagrad": Adagrad(lr=config.lr),
        "rmsprop": RMSprop(lr=config.lr),
        "adam": Adam(lr=config.lr),
        "nadam": Nadam(lr=config.lr)
    }[config.optimizer]

    model = Sequential()
    model.add(Dense(trainX.shape[1],
                    config.hidden_layer_size,
                    activation=config.activation,
                    initialization=config.weight_init))

    for _ in range(config.num_hidden_layers):
        model.add(Dense(config.hidden_layer_size,
                        config.hidden_layer_size,
                        activation=config.activation,
                        initialization=config.weight_init))

    model.add(Dense(config.hidden_layer_size,
                    trainy.shape[1],
                    activation="softmax" if config.loss_function == "categorical_crossentropy" else "identity",
                    initialization=config.weight_init))

    model.compile(loss=config.loss_function,
                  optimizer=optimizer,
                  weight_decay=config.l2)

    model.fit(trainX,
              trainy,
              batch_size=config.batch_size,
              epochs=config.epochs,
              validation_data=(valX, valy),
              wandb_log=True)
    
    loss, acc = model.evaluate(testX, testy, batch_size=config.batch_size)

    wandb.log({        
        "test_loss": loss,
        "test_accuracy": acc,     
    })

    run.finish()


In [ ]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'activation': {
            'values': ['leaky_relu']
        },
        'loss_function': {
            'values': ['categorical_crossentropy', 'mean_squared_error']
        },
        'lr': {
            'values': [1e-2]
        },
        'batch_size': {
            'values': [64]
        },
        'hidden_layer_size': {
            'values': [128]
        },
        'epochs': {
            'values': [15]
        },
        'num_hidden_layers': {
            'values': [3]
        },
        'optimizer': {
            'values': ['adagrad']
        },
        'l2': {
            'values': [5e-2]
        },
        'weight_init': {
            'values': ['xavier']
        }
    }
}


In [ ]:
wandb.login()
sweep_id = wandb.sweep(sweep_config, project='IITM-CS6910-Projects',entity="cs21m070_cs21m022")
wandb.agent(sweep_id, function=build_and_run_model)